# Data imports


In [1]:
import TNSPC as tp
import torch

device = torch.device('cpu')
#to use cude need to install the package independently
#device = torch.device('cuda')
torch.manual_seed(0)

### Importing mnist and saving to arff file

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

# Step 1: Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data[:10000]
X = X / 255.0
y = mnist.target[:10000].astype(int).to_numpy()

# Step 2: Convert to Pandas DataFrame
print("Converting dataset to DataFrame...")
df = pd.DataFrame(X, columns=[f'pixel{i}' for i in range(1, X.shape[1]+2)])
#df['class'] = y  # Append labels as the last column


# Step 3: Write to ARFF file manually
def write_arff(df, filename, relation_name, class_labels):
    with open(filename, 'w') as f:
        # Write relation name
        f.write(f"@relation {relation_name}\n\n")
        
        # Write attributes
        for column in df.columns[:-1]:
            f.write(f"@attribute {column} numeric\n")
        f.write(f"@attribute class_labels {{{','.join(map(str, class_labels))}}}\n\n")
        # Write data
        f.write("@data\n")
        for row in df.values:
            f.write(','.join(map(str, row)) + '\n')

output_file = 'mnist_2.arff'
write_arff(df, output_file, relation_name='mnist', class_labels=range(10))

print(f"ARFF file saved to {output_file}")


Loading MNIST dataset...
Converting dataset to DataFrame...
ARFF file saved to mnist_2.arff


### Loading in the data from the arff file

In [17]:
import TNSPC as tp
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from TNSPC.utils.utils import drawnow, arrange_figs
from TNSPC.layers.leaves import *
from TNSPC.layers.tn import *
from TNSPC.layers.spn import *
from TNSPC.layers.TNSPCclass import *
from TNSPC.utils.model import *


file_path = "mnist.arff"

df, attribute_mapping, leaves_types = tp.utils.data.hetdata(file_path, reorder=True)
print(df.values.shape)

frac_train = int(0.8*len(df))
frac_test = len(df) - int(0.8*len(df))
Xtrain = torch.tensor(df.values[:frac_train]).float()

print(Xtrain.shape)
Xtest = torch.tensor(df.values[frac_train:]).float()
print(y[:frac_train].shape)
Ytrain = torch.tensor(y[:frac_train]).int()
Ytest = torch.tensor(y[frac_train:]).int()

#%% Fit model
N = frac_train

(10000, 784)
torch.Size([8000, 784])
(8000,)


# PC model

### Model structure

In [19]:
from TNSPC.layers.leaves import NormalLeaf, CategoricalLeaf
from TNSPC.layers.TNSPCclass import MultiLeaf

# Hyperparameters
T = 1 # Number of tracks
C = 10  # Number of channels
V = 784  # Number of input variables (e.g., pixels)
classes = 10  # Number of output classes (e.g., digits 0-9)

# Define individual leaf distributions
input_leaf = BernoulliLeaf(T, V, C)         # Gaussian for pixel inputs
output_leaf = CategoricalLeaf(T, 1, C, 10)  # Categorical for class labels


# Combine them into a MultiLeaf
multi_leaf = MultiLeaf(input_leaf, output_leaf)

print(multi_leaf[1])

network = Sequential(multi_leaf,
                     Weightsum(T, V + 1, C)) 

model = tp.utils.model.CP_(T, V, C, multi_leaf).to(device)

print(network)

CategoricalLeaf()
Sequential(
  (0): MultiLeaf(
    (0): BernoulliLeaf()
    (1): CategoricalLeaf()
  )
  (1): Weightsum()
)


In [20]:
Xtrain = Xtrain > 0

In [21]:

concatenated = torch.cat((Xtrain.float(), Ytrain[:,None]), dim =1)
concatenated.shape
concatenated[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 

In [22]:
x = concatenated
for i, layer in enumerate(network):
    x = layer(x)
    print(f"After layer {i} ({layer.__class__.__name__}): {x.shape}")

print()

After layer 0 (MultiLeaf): torch.Size([8000, 1, 785, 10])
After layer 1 (Weightsum): torch.Size([8000])



In [23]:
network.train()
tp.utils.model.train(network, [concatenated], device = device, epochs = 10, show_progress=True)

Epoch 10, Avg P: -179.6649: 100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


In [24]:
Xtest = Xtest > 0
empty_y = torch.zeros(Xtest.shape[0])
empty_y[:] = float('nan')
concatenated_test = torch.cat((Xtest.float(), empty_y[:,None]), dim =1)


In [25]:
leaves_types = ["N"] * 784 + ["C"]
print(leaves_types)

imp_func = lambda model, batch, device: tp.utils.impute.impute(model, batch, leaves_types, device = device)

#We can now impute the data introducing some randomn missing values

p = 0.5
mXtest = torch.clone(concatenated_test)

b_mXtest = tp.utils.data.batch(mXtest, 200)

Ximputed = tp.utils.impute.b_impute(model, b_mXtest , imp_func, device)

predicted_label = Ximputed[:,-1]


['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',

In [26]:
torch.sum(predicted_label == Ytest)

tensor(1247)

# CNN model

In [50]:

# Create Dataset objects for training and validation
train_dataset = torch.utils.data.TensorDataset(Xtrain.view(-1, 1, 28, 28), Ytrain)
val_dataset = torch.utils.data.TensorDataset(Xtest.view(-1, 1, 28, 28), Ytest)

# Create DataLoader objects for loading data in batches
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [51]:
class PrintSize(nn.Module):
    """Utility module to print current shape of a Tensor in Sequential, only at the first pass."""

    first = True

    def forward(self, x):
        if self.first:
            print(f"Size: {x.size()}")
            self.first = False
        return x


class Model(nn.Module):
    def __init__(self, num_classes = 10):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

        self.print_size = PrintSize()

    def forward(self, x):

      x = self.dropout(self.pool(F.relu(self.conv1(x.float()))))
      x = self.dropout(self.pool(F.relu(self.conv2(x))))

      x = self.print_size(x)

      x = x.view(x.size(0), -1)
      x = self.dropout(F.relu(self.fc1(x)))

      x = self.fc2(x)

      return x


model = Model(10)
device = torch.device('cpu')  # use cuda or cpu
model.to(device)
print(model)

Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (print_size): PrintSize()
)


In [53]:
# Initialize the network, loss function, and optimizer
import torch.optim as optim


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()
            # Convert labels to Long type
        labels = labels.long()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation step
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.long()

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {100 * correct / total:.2f}%")


Epoch 1/10, Training Loss: 0.2681, Validation Loss: 0.2011, Validation Accuracy: 93.75%
Epoch 2/10, Training Loss: 0.1794, Validation Loss: 0.1738, Validation Accuracy: 94.35%
Epoch 3/10, Training Loss: 0.1399, Validation Loss: 0.1554, Validation Accuracy: 95.20%
Epoch 4/10, Training Loss: 0.1173, Validation Loss: 0.1468, Validation Accuracy: 95.10%
Epoch 5/10, Training Loss: 0.0940, Validation Loss: 0.1293, Validation Accuracy: 95.55%
Epoch 6/10, Training Loss: 0.0869, Validation Loss: 0.1136, Validation Accuracy: 96.35%
Epoch 7/10, Training Loss: 0.0739, Validation Loss: 0.1059, Validation Accuracy: 96.15%
Epoch 8/10, Training Loss: 0.0610, Validation Loss: 0.1188, Validation Accuracy: 96.55%
Epoch 9/10, Training Loss: 0.0632, Validation Loss: 0.1143, Validation Accuracy: 95.90%
Epoch 10/10, Training Loss: 0.0524, Validation Loss: 0.1162, Validation Accuracy: 96.30%


In [32]:
import torch.optim as optim


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [34]:
# Test the forward pass with dummy data
out = model(torch.randn(2, 1, 32, 32, device=device))
print("Output shape:", out.size())
print(f"Output logits:\n{out.detach().cpu().numpy()}")
print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")

Size: torch.Size([2, 64, 8, 8])
Output shape: torch.Size([2, 10])
Output logits:
[[-0.04889199  0.13756183 -0.07870828 -0.09886788  0.10629128 -0.05245399
   0.05378672 -0.01691845  0.05210052 -0.13087699]
 [-0.02653929 -0.07995768 -0.15032569  0.11647131  0.3056151   0.13593486
  -0.23364216  0.14802805  0.02721573 -0.16597223]]
Output probabilities:
[[0.09560831 0.11520505 0.0927997  0.09094763 0.11165827 0.09526835
  0.10594694 0.09871464 0.10576845 0.08808258]
 [0.0953796  0.09041826 0.0842744  0.11004344 0.1329561  0.11220627
  0.0775375  0.11357143 0.10064702 0.08296607]]


In [ ]:
from torch.utils.data import  DataLoader


train_loader = DataLoader(Train_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=False)


NameError: name 'train_set' is not defined

In [ ]:
num_epochs = 2
validation_every_steps = 500

step = 0
model.train()

train_accuracies = []
valid_accuracies = []

for epoch in range(num_epochs):

    train_accuracies_batches = []

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        output = model(inputs)
        loss = loss_fn(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Increment step counter
        step += 1

        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy(targets, predictions))

        if step % validation_every_steps == 0:

            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))

            train_accuracies_batches = []

            # Compute accuracies on validation set.
            valid_accuracies_batches = []
            with torch.no_grad():
                model.eval()
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    output = model(inputs)
                    loss = loss_fn(output, targets)

                    predictions = output.max(1)[1]

                    # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    valid_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

                model.train()

            # Append average validation accuracy to list.
            valid_accuracies.append(np.sum(valid_accuracies_batches) / len(test_set))

            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
            print(f"             test accuracy: {valid_accuracies[-1]}")

print("Finished training.")